In [1]:
# This script is for RF, socio variables
# Take Dataset3 BUF for example

In [2]:
# Packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import statsmodels.api as sm

In [3]:
X_socio_test_passed_1 = pd.read_csv("../Data/01 Processed Dataset/Socio_BUF.csv") #input
X_socio_test_passed = X_socio_test_passed_1.set_index('GEOID')
y = X_socio_test_passed.pop('obesity_cr')

In [4]:
X_socio_test_passed.head(10)

,% Black,% Ame Indi and AK Native,% Asian,% Nati Hawa and Paci Island,% Hispanic or Latino,% male,% married,% age 18-29,% age 30-39,% age 40-49,...,% age >=60,% <highschool,median income,% unemployment,% below poverty line,% food stamp/SNAP,median value units built,median year units built,% renter-occupied housing units,population density
GEOID,,,,,,,,,,,,,,,,,,,,,
36029000500,0.062950,0.000000,0.000000,0.0,0.153477,0.432854,0.361005,0.118106,0.126499,0.083933,...,0.220624,0.228101,26605,0.063401,0.414311,0.331218,50800,1939,0.376904,266.853532
36029000700,0.004076,0.000000,0.007337,0.0,0.029076,0.454891,0.472657,0.165761,0.135870,0.111685,...,0.254891,0.067675,71944,0.047530,0.062044,0.100134,125300,1944,0.242323,3437.581736
36029000900,0.005390,0.000000,0.005390,0.0,0.180763,0.484245,0.391390,0.207711,0.227612,0.093284,...,0.176202,0.100644,52768,0.078431,0.097015,0.135734,93000,1939,0.445060,4384.936326
36029001000,0.092095,0.034719,0.016137,0.0,0.055094,0.515729,0.427102,0.193154,0.168867,0.082967,...,0.193480,0.122189,49566,0.081107,0.207792,0.317936,79400,1939,0.520183,2932.488101
36029001100,0.057063,0.015310,0.032359,0.0,0.105428,0.525052,0.324919,0.186152,0.134308,0.092206,...,0.217815,0.170852,37886,0.071886,0.213184,0.239050,74300,1939,0.543430,1885.467988
36029001600,0.521115,0.000000,0.069701,0.0,0.162362,0.577286,0.314763,0.218532,0.123821,0.113161,...,0.137761,0.203205,16605,0.219828,0.600084,0.627134,31900,1939,0.735849,1240.066503
36029001700,0.276382,0.003350,0.026801,0.0,0.218314,0.490229,0.307042,0.198772,0.151312,0.061977,...,0.228364,0.316742,17360,0.176084,0.552764,0.510417,35000,1939,0.609375,1536.402707
36029003700,0.815896,0.000000,0.043169,0.0,0.036059,0.489335,0.335347,0.154393,0.161503,0.094718,...,0.173438,0.250763,23792,0.143048,0.422284,0.533619,36700,1939,0.708435,3581.790895
36029006301,0.099455,0.000000,0.014918,0.0,0.061094,0.479754,0.425254,0.304523,0.210987,0.098745,...,0.155577,0.094787,57700,0.017667,0.132573,0.153991,233000,1939,0.572300,5830.328199


In [5]:
len(X_socio_test_passed), len(X_socio_test_passed.columns)

(77, 21)

In [6]:
y.head()

GEOID
36029000500    38.6
36029000700    27.7
36029000900    30.8
36029001000    32.4
36029001100    33.4
Name: obesity_cr, dtype: float64

In [7]:
def standarize_data(data, stats):
    return (data - stats['mean'])/ stats['std']

In [8]:
# Random forest 
y_rf_socio_predict = []
y_true = []

ten_fold = KFold(n_splits=10, shuffle=True, random_state=42)

avg_r_squared = 0

for train_index, test_index in ten_fold.split(X_socio_test_passed):
    print("TEST:", test_index)

    X_train, X_test = X_socio_test_passed.iloc[train_index], X_socio_test_passed.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    training_stat = X_train.describe().transpose()
    scaled_X_train = standarize_data(X_train, training_stat)
    scaled_X_test = standarize_data(X_test, training_stat)

    rf = RandomForestRegressor(bootstrap=False, n_estimators=160, max_features="sqrt", random_state=42) # input
    rf.fit(scaled_X_train, y_train)

    this_y_predict = rf.predict(scaled_X_test).flatten()

    y_rf_socio_predict = y_rf_socio_predict + this_y_predict.tolist()
    y_true = y_true + y_test.tolist()
    
    avg_r_squared += r2_score(y_test, this_y_predict)

TEST: [ 0  4 10 35 45 47 53 66]
TEST: [12 18 28 33 50 58 68 74]
TEST: [ 5  9 16 22 30 34 39 40]
TEST: [ 7 31 44 49 54 62 65 73]
TEST: [ 3 13 19 25 42 55 64 76]
TEST: [ 6  8 17 36 38 56 61 70]
TEST: [15 24 26 27 41 46 48 75]
TEST: [11 32 43 57 59 69 72]
TEST: [ 1 21 29 37 52 63 67]
TEST: [ 2 14 20 23 51 60 71]


In [9]:
rf_socio_rmse = mean_squared_error(y_true , y_rf_socio_predict, squared=False) #False means return RMSE value
rf_socio_r2 = r2_score(y_true, y_rf_socio_predict)
avg_r_squared = avg_r_squared/10

In [10]:
# sociodemographic - estimators
print("RMSE of the RF model with sociodemographic predictors: "+str(rf_socio_rmse))
print("R2 of the RF model with sociodemographic predictors: "+str(rf_socio_r2)) # For R2, I took this one.
print("R2 of the RF model with sociodemographic predictors (avg): "+str(avg_r_squared))

RMSE of the RF model with sociodemographic predictors: 2.5136176652880935
R2 of the RF model with sociodemographic predictors: 0.8693876881351768
R2 of the RF model with sociodemographic predictors (avg): 0.8556503148905735
